#### <font color="#03997A"> Diplomado en Ciencia de datos UNAM </font>

##### <font color="#03997A"> Dr: Alejandro Pimentel  </font>

##### <font color="#03997A"> Modulo 8 Introducción al Deep Learning </font>

##### <font color="#03997A"> Ejercicio 4 CNN  Alumno: Ibarra Ramírez Sergio </font>

In [16]:
import tensorflow as tf
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import normalize
from sklearn.preprocessing import OneHotEncoder

X,y = fetch_openml('mnist_784', as_frame=False, return_X_y=True)

X = normalize(X).reshape(-1,28,28,1)

X_train = X[:60000]
X_test = X[60000:]
y_train = y[:60000]
y_test = y[60000:]

codificador = OneHotEncoder(handle_unknown='ignore',sparse=False)
y_train = codificador.fit_transform(y_train.reshape(-1,1))
y_test = codificador.transform(y_test.reshape(-1,1))

X_train = tf.constant(X_train, dtype=tf.float32)
X_test = tf.constant(X_test, dtype=tf.float32)
y_train = tf.constant(y_train, dtype=tf.float32)
y_test = tf.constant(y_test, dtype=tf.float32)

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
batch_size = 100 # Hiperparámetro
train_dataset = train_dataset.shuffle(batch_size).batch(batch_size)

c:\Users\Sergio\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\datasets\_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(
c:\Users\Sergio\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [17]:
b = tf.Variable(tf.random.normal([10]))
W = tf.Variable(tf.random.normal([1568,10]))

filtros1 = tf.Variable(tf.random.normal([4,4,1,16])) # (alto,ancho,canales,num_filtros)
b_conv1 = tf.Variable(tf.random.normal([16]))

filtros2 = tf.Variable(tf.random.normal([4,4,16,32])) # (alto,ancho,canales,num_filtros)
b_conv2 = tf.Variable(tf.random.normal([32]))

optimizador = tf.keras.optimizers.experimental.SGD(learning_rate=0.01)

In [18]:
# Training loop

epochs = 10

for _ in range(epochs):
  loss_epoch = 0
  for x,y in train_dataset:
    with tf.GradientTape() as gt:

      salida_conv1 = tf.nn.conv2d(x,filtros1,[1,2,2,1],"SAME")
      salida_conv1 += b_conv1
      salida_activacion1 = tf.nn.relu(salida_conv1) # Capa no lineal intermedia

      salida_conv2 = tf.nn.conv2d(salida_activacion1,filtros2,[1,2,2,1],"SAME")
      salida_conv2 += b_conv2
      #salida_activacion2 =tf.nn.relu(salida_conv2) # necesita el relu?

      salidaPlana = tf.reshape(salida_conv2,[-1,1568])

      operacion_matricial = tf.matmul(salidaPlana,W)+b # Primera capa

      x_ent = tf.nn.softmax_cross_entropy_with_logits(
          logits = operacion_matricial,
          labels = y
      )

      loss = tf.reduce_mean(x_ent)
      optimizador.minimize(loss,[W,b,filtros1,b_conv1, filtros2, b_conv2],gt) # Poderosa

    loss_epoch += loss/len(train_dataset)
  print(loss_epoch)

tf.Tensor(14.883716, shape=(), dtype=float32)
tf.Tensor(3.3829992, shape=(), dtype=float32)
tf.Tensor(2.8502724, shape=(), dtype=float32)
tf.Tensor(2.6319265, shape=(), dtype=float32)
tf.Tensor(2.507591, shape=(), dtype=float32)
tf.Tensor(2.4456027, shape=(), dtype=float32)
tf.Tensor(2.4044218, shape=(), dtype=float32)
tf.Tensor(2.384183, shape=(), dtype=float32)
tf.Tensor(2.3456023, shape=(), dtype=float32)
tf.Tensor(2.3303554, shape=(), dtype=float32)


In [19]:
salida_conv1 = tf.nn.conv2d(X_test,filtros1,[1,2,2,1],"SAME")
salida_conv1 += b_conv1
salida_activacion1 = tf.nn.relu(salida_conv1) # Capa no lineal intermedia
salida_conv2 = tf.nn.conv2d(salida_activacion1,filtros2,[1,2,2,1],"SAME")
salida_conv2 += b_conv2
#salida_activacion2 =tf.nn.relu(salida_conv2) # necesita el relu?

salidaPlana = tf.reshape(salida_conv2,[-1,1568])

operacion_matricial = tf.matmul(salidaPlana,W)+b # Primera capa

probs = tf.nn.softmax(operacion_matricial)

max_preds = tf.argmax(probs,1)
max_trues = tf.argmax(y_test,1)

numCorrectos = tf.equal(max_preds,max_trues)
acc = tf.reduce_mean(tf.cast(numCorrectos,tf.float32)) # Tengo que convertirlo porque el original es booleano
print(acc)

tf.Tensor(0.1675, shape=(), dtype=float32)
